In [1]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam
from VariationalBayes.NormalParams import MVNParam, UVNParam
from VariationalBayes.GammaParams import GammaParam

# import math

from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np
import autograd.numpy.random as npr

import copy
import scipy
from scipy import optimize
from scipy import stats

In [2]:
# This should go in the main library.
from VariationalBayes.Parameters import set_free_offset, get_free_offset

class UVNParamVector(object):
    def __init__(self, name, length, min_var=0.0):
        self.name = name
        self.mean = VectorParam(name + '_mean', length)
        self.var = VectorParam(name + '_var', length, lb=min_var)
        self.__free_size = self.mean.free_size() + self.var.free_size()
    def __str__(self):
        return self.name + ':\n' + str(self.mean) + '\n' + str(self.var)
    def names(self):
        return self.mean.names() + self.var.names()
    def e(self):
        return self.mean.get()
    def e_outer(self):
        mean = self.mean.get() ** 2 + self.var.get()
    def set_free(self, free_val):
        if free_val.size != self.__free_size: \
            raise ValueError('Wrong size for UVNParam ' + self.name)
        offset = 0
        offset = set_free_offset(self.mean, free_val, offset)
        offset = set_free_offset(self.var, free_val, offset)
    def get_free(self):
        vec = np.empty(self.__free_size)
        offset = 0
        offset = get_free_offset(self.mean, vec, offset)
        offset = get_free_offset(self.var, vec, offset)
        return vec
    def free_size(self):
        return self.__free_size

In [3]:
N = 100     # observations per group
K = 2       # dimension of regressors
NG = 5      # number of groups

# Build an object to contain a variational approximation to a K-dimensional multivariate normal.
glmm_par = ModelParamsDict()

glmm_par.push_param(UVNParam('mu'))
glmm_par.push_param(GammaParam('tau'))
glmm_par.push_param(MVNParam('beta', K))
glmm_par.push_param(UVNParamVector('u', NG))

glmm_par['mu'].mean.set(0.1)
glmm_par['mu'].var.set(1.0)

glmm_par['tau'].shape.set(2.1)
glmm_par['tau'].rate.set(2.1)

glmm_par['beta'].mean.set(np.full(K, 0.))
glmm_par['beta'].cov.set(0.2 * np.eye(K))

glmm_par['u'].mean.set(np.full(NG, 0.))
glmm_par['u'].var.set(np.full(NG, 0.1))
    
print glmm_par

ModelParamsList:
	mu:
mu_mean: 0.1
mu_var: 1.0
	tau:
tau_shape: 2.1
tau_rate: 2.1
	beta:
beta_mean:
[ 0.  0.]
beta_cov:
[[ 0.2  0. ]
 [ 0.   0.2]]
	u:
u_mean:
[ 0.  0.  0.  0.  0.]
u_var:
[ 0.1  0.1  0.1  0.1  0.1]


In [4]:
# Generate data

def Logistic(u):
    return np.exp(u) / (1 + np.exp(u))

NObs = NG * N
true_beta = np.random.rand(K) - 0.5
true_mu = 0.2
true_tau = 4.0
true_u = np.random.normal(true_mu, 1 / np.sqrt(true_tau), NG)

x_mat = np.random.random(K * NObs).reshape(NObs, K) - 0.5
y_g_vec = [ g for g in range(NG) for n in range(N) ]
true_rho = Logistic(np.matmul(x_mat, true_beta) + true_u[y_g_vec])
y_vec = np.random.random(NObs) < true_rho


In [5]:
def DataLogLikelihood(x_mat, y_vec, e_beta, e_beta_outer, e_u, var_u, std_draws):
    rho_mean = e_u + np.matmul(x_mat, e_beta)
    rho_sd = np.sqrt(var_u + np.einsum('nk,kj,nj->n', x_mat, e_beta_outer, x_mat))
    z = np.einsum('i,j->ij', rho_sd, std_draws) + np.expand_dims(rho_mean, 1)

    # The sum is over observations and draws, so dividing by the draws size
    # gives the sum of sample expectations over the draws.
    # log(1 - p) = log(1 / (1 + exp(u))) = -log(1 + exp(u))
    logit_term = -np.sum(np.log1p(np.exp(z))) / std_draws.size
    y_term = np.sum(y_vec * rho_mean)
    return y_term + logit_term


def RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau):
    return -0.5 * e_tau * np.sum((e_mu - e_u) ** 2 + var_mu + var_u) \
           - 0.5 * e_log_tau * len(e_u)

def UnivariateNormalEntropy(var_obs):
    return 0.5 * np.sum(np.log(var_obs))

def MultivariateNormalEntropy(cov_obs):
    sign, logdet = np.linalg.slogdet(cov_obs)
    assert sign > 0
    return 0.5 * logdet

def GammaEntropy(shape, rate):
    return np.sum(shape - np.log(rate) + scipy.special.gammaln(shape) + \
                  (1 - shape) * scipy.special.digamma(shape))

def MVNPrior(prior_mean, prior_info, e_obs, cov_obs):
    obs_diff = e_obs - prior_mean
    return -0.5 * (np.dot(obs_diff, np.matmul(prior_info, obs_diff)) + \
                   np.trace(np.matmul(prior_info, cov_obs)))

def UVNPrior(prior_mean, prior_info, e_obs, var_obs):
    return -0.5 * (prior_info * ((e_obs - prior_mean) ** 2 + var_obs))


def Elbo(y_vec, x_mat, y_g_vec, glmm_par_elbo, std_draws):
    e_beta = glmm_par['beta'].mean.get()
    cov_beta = glmm_par['beta'].cov.get()
    e_beta_outer = glmm_par['beta'].e_outer()
    
    e_u = glmm_par['u'].mean.get()
    var_u = glmm_par['u'].var.get()
    
    e_mu = glmm_par['mu'].mean.get()
    var_mu = glmm_par['mu'].var.get()
    
    e_tau = glmm_par['tau'].e()
    e_log_tau = glmm_par['tau'].e_log()

    ll = \
        DataLogLikelihood(x_mat, y_vec, e_beta, e_beta_outer,
                          e_u[y_g_vec], var_u[y_g_vec], std_draws) + \
        RandomEffectLogLikelihood(e_u, var_u, e_mu, var_mu, e_tau, e_log_tau)

    K = len(e_beta)
    beta_prior_info = 0.01 * np.eye(K)
    beta_prior_mean = np.full(K, 0.)
    e_log_prior = \
        MVNPrior(beta_prior_mean, beta_prior_info, e_beta, cov_beta) + \
        UVNPrior(0., 0.01, e_mu, var_mu)
    # TODO: add the other priors.
        
    entropy = \
        UnivariateNormalEntropy(var_mu) + \
        MultivariateNormalEntropy(cov_beta) + \
        UnivariateNormalEntropy(var_u) + \
        GammaEntropy(glmm_par['tau'].shape.get(), glmm_par['tau'].rate.get())

    return ll + e_log_prior + entropy


class KLWrapper():
    def __init__(self, glmm_par, x_mat, y_vec, y_g_vec, num_draws):
        self.__glmm_par_ad = copy.deepcopy(glmm_par)
        self.x_mat = x_mat
        self.y_vec = y_vec
        self.y_g_vec = y_g_vec
        draw_spacing = 1 / float(num_draws + 1)
        target_quantiles = np.linspace(draw_spacing, 1 - draw_spacing, num_draws)
        self.std_draws = scipy.stats.norm.ppf(target_quantiles)
    def Eval(self, free_par_vec, verbose=False):
        self.__glmm_par_ad.set_free(free_par_vec)
        kl = -Elbo(self.y_vec, self.x_mat, self.y_g_vec,
                   self.__glmm_par_ad, std_draws=self.std_draws)
        if verbose: print kl
        return kl
    
    # Return a posterior moment of interest as a function of
    # unconstrained parameters.  In this case it is a bit silly,
    # but in full generality posterior moments may be a complicated
    # function of moment parameters.
    def GetMoments(self, free_par_vec):
        self.__glmm_par_ad.set_free(free_par_vec)
        return self.__glmm_par_ad['beta'].mean.get()



In [6]:
kl_wrapper = KLWrapper(glmm_par, x_mat, y_vec, y_g_vec, 10)
KLGrad = grad(kl_wrapper.Eval)
KLHess = hessian(kl_wrapper.Eval)
MomentJacobian = jacobian(kl_wrapper.GetMoments)
KLHessVecProd = hessian_vector_product(kl_wrapper.Eval)  
free_par_vec = glmm_par.get_free()
kl_wrapper.Eval(free_par_vec)


360.39495753257819

In [7]:
import timeit

time_num = 10

print 'Function time:'
print timeit.timeit(lambda: kl_wrapper.Eval(free_par_vec), number=time_num) / time_num

print 'Grad time:'
print timeit.timeit(lambda: KLGrad(free_par_vec), number=time_num) / time_num

print 'Hessian vector product time:'
print timeit.timeit(lambda: KLHessVecProd(free_par_vec, free_par_vec + 1), number=time_num) / time_num

# so slow
# print 'Hessian time:'
# print timeit.timeit(lambda: KLHess(free_par_vec), number=time_num) / time_num


Function time:
0.000590014457703
Grad time:
0.000948405265808
Hessian vector product time:
0.00117259025574


/home/rgiordan/Documents/git_repos/autograd/autograd/core.py:16: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


In [8]:
import cProfile

profile = False
if profile:
    cProfile.run('kl_wrapper.Eval(free_par_vec)', '/tmp/cprofilestats_func.prof')
    cProfile.run('KLHess(free_par_vec)', '/tmp/cprofilestats_hess.prof')
    cProfile.run('KLGrad(free_par_vec)', '/tmp/cprofilestats_grad.prof')

# A better way to visualize this https://jiffyclub.github.io/snakeviz/
# snakeviz /tmp/cprofilestats_hess.prof

In [9]:
# Set initial values.
xtx = np.matmul(x_mat.T, x_mat)
mu_reg = np.linalg.solve(xtx, np.matmul(x_mat.T, y_vec))

mvn_par['e_mu'].set(mu_reg)
mvn_par['var_mu'].set(np.full(K, 1.))
init_par_vec = mvn_par.get_free()

NameError: name 'mvn_par' is not defined

In [ ]:
# Optimize.
# print 'Running BFGS'
# vb_opt_bfgs = optimize.minimize(
#     lambda par: kl_wrapper.Eval(par, verbose=True), init_par_vec,
#     method='bfgs', jac=KLGrad, tol=1e-2)

print 'Running Newton Trust Region'
vb_opt = optimize.minimize(
    lambda par: kl_wrapper.Eval(par, verbose=True),
    init_par_vec, method='trust-ncg', jac=KLGrad, hessp=KLHessVecProd)
mvn_par_opt = copy.deepcopy(mvn_par)
mvn_par_opt.set_free(vb_opt.x)
print 'Done.'

In [ ]:
# The mean parameters match, as expected.
from ggplot import *
import pandas as pd
%matplotlib inline

plot_df = pd.DataFrame({ 'opt': mvn_par_opt['e_mu'].get(), 'true': true_mu })
ggplot(plot_df, aes(x='opt', y='true')) + geom_point() + geom_abline(slope=1, intercept=0)

In [ ]:
# LRVB with conjugate gradient
from scipy.sparse.linalg import LinearOperator
import sys

# The we will actually compute Hess^1 * moment_jac.T, leading to perhaps confusing
# naming of "columns".  
moment_jac = MomentJacobian(vb_opt.x)
ObjHessVecProdLO = LinearOperator((vb_opt.x.size, vb_opt.x.size), lambda par: KLHessVecProd(vb_opt.x, par))
# print moment_jac.T.shape
# print ObjHessVecProdLO.shape
# cg_res, info = scipy.sparse.linalg.cg(ObjHessVecProdLO, moment_jac.T)

lrvb_term = np.full(moment_jac.T.shape, float('nan'))
for col in range(moment_jac.shape[0]):
    sys.stdout.write('.')
    sys.stdout.flush()
    cg_res, info = scipy.sparse.linalg.cg(ObjHessVecProdLO, moment_jac[col, :])
    assert info == 0
    lrvb_term[:, col] = cg_res

print 'all done dude'

In [ ]:
# Slow:
kl_hess = KLHess(vb_opt.x)

In [ ]:
mu_cov_hess = np.matmul(moment_jac, np.linalg.solve(kl_hess, moment_jac.T))
mu_cov = np.matmul(moment_jac, lrvb_term)
print np.diag(mu_cov)
print np.diag(mu_cov_hess)
print mvn_par_opt['var_mu']